In [13]:
import numpy as np


class fsa:
    def __init__(self, initial={1}, final=set(), transitions= {}):
        """Class for finite-state automata.
        
        Arguments
        ---------
        initial: int or string
            name of initial state
            default: {1}
        final: set
            set of final states
            default: empty set
        transitions: dict
            dictionary of the form {current_state: {arc_label: {new_state}}}
            default: empty dictionary
        *E* is an empty string
        """
        self.I = initial
        self.F = final
        self.T = transitions
        
    def eps_free(self):
        """converts a NDFSA to a DFSA (kinda)
        basically takes care of empty strings
        current state = where a transition can come from
        arc_label = how you get from one state to another
        new_states= where we land on the end of the arc_label, is a state containing >= 1 item
        could be the current_state (repeating)"""
        #find any empty strings
        #while there is an E in T then keep running the for loop
        r = "it doesn't really matter what does here"
        for item in self.T: 
            current_state = item
            arc_label = self.T[current_state]
            for arc in arc_label:
                if arc == "*E*":
                    r = "YES"
                    break
                
        while r == ("YES" or "YES2"):
        #this is where things get a little crazy
            for item in self.T: 
                current_state = item
                arc_label = self.T[current_state]
                for arc in arc_label:
                    if arc == "*E*": #could have two Es from one source
                        new_states = [item for item in arc_label[arc]] #this is the destination(s) for the empty string
                        for state in new_states:                          #could be current state, one state, or >1 state
                            if state in self.F:
                    #if a "to" is a F then FS must also be a F
                                self.F.add(current_state) #if the CS is already in F this won't change anything
                            goin = self.T.get(state) #can be a dictionary or None\
                            if goin != None:
                                new_dict = {**self.T[current_state], **goin}
                                self.T[current_state] = new_dict
                                del self.T[current_state]["*E*"]
            #basically, we want to find any more Es. However, we also don't want it to end too early and that's why we had to make "yes2"
            for item in self.T: 
                current_state = item
                arc_label = self.T[current_state]
                for arc in arc_label:
                    if arc == "*E*":
                        r = "YES"
                        break
                    else:
                        r = "YES2"
            if r == "YES2":
                r = "NO"
                                
        return self.T, self.F
     
    
    def mm_con(self):
        """Method to make a FSA work with matrix multiplication
        states = a list of the states
        n = number to states (integer)
        ni = new initial, array
        nf = new final, array
        nt = new transition, dictionary in the form {Tranisition: array}
        returns a tuple in the form (ni, nf, nt, n)
        """
        #states compiles all the states so we can count them- adding the transitions then the final states if not in trans
        self.eps_free()
        states = [item for item in self.T]
        lst = []
        for item in self.F:
            if item not in states:
                states.append(item)
        for state in self.T:
            cs = self.T[state]
            for thing in cs:
                lst.append(thing)
        n = len(states)
        for item in lst:
            if item not in states:
                n = n+1
        #creates a "blank" array of zeros for ni and nf
        ni = np.zeros( (1, n) )
        nf = np.zeros( (n, 1) )
        #sets the column that corresponds to inital state to 1
        for num in self.I:
            ni[0][num-1] = 1
        #create a new dictionary for the transition arrays
        nt = {}
        #creates the array for final by replacing the 0 with a 1 in the correct place
        for num in self.F:
            nf[num-1][0] = 1
        #creates the arrays for transitions
        #creates a new word "blank" array then sets the correct value to 1
        #basically just added another for loop to "access" the new dictionary
        #if an array for that word already exists they are added
        for item in self.T:
            cs = self.T[item]
            for word in cs:
                css = cs.get(word)
                for numb in css:
                    nw = np.zeros( (n, n) )
                    nw[item-1][numb-1] = 1
                    if np.any(nt.get(word)):
                        nt[word] = nt[word] + nw
                    else:
                        nt[word] = nw
        #returns a tuple of all the values needed for accepts
        return ni, nf, nt, n
        
    def n_accepts(self, sentence):
        """a new accept method
        sentence in the form of a tokenized list (like original accept method)
        conversion runs the convert method
        cs = current state, set to the initial array
        nt, nf, n = setting these valoue to their corresponding values in the returned tuple
        """
        conversion = self.mm_con()
        cs = conversion[0]
        nt = conversion[2]
        nf = conversion[1]
        n = conversion[3]
        #multiply the inital by the first transition, then by the next transition
        #the sign makes it so 0=0 and any pos. number=1
        for word in sentence:
            cs = cs.dot(nt.get(word))
            cs = np.sign(cs)
        #multiply the current state by the final
        cs = cs.dot(nf)
        return True if cs >= [1] else False

In [14]:
#example1 = fsa(initial={0}, final={2}, transitions={0:{"a":{1}}, 1:{"b":{2}}, 2:{"c":{2}}})
#print("ex 1 a:", example1.n_accepts("a"))
#print("ex 1 ab:", example1.n_accepts("ab"))
#print("ex 1 abc:", example1.n_accepts("abc"))
example2 = fsa(initial = {0}, final = {2}, transitions = {0: {"a":{1}}, 1:{"b":{2},"*E*":{2}}, 2:{"d":{3},"c":{0}, "*E*":{1}}})
#print("ex 2 a:", example2.n_accepts("a"))
#print("ex 2 ab:", example2.n_accepts("ab"))
#print("ex 2 abc:", example2.n_accepts("abc"))
#print("ex 2 abca:", example2.n_accepts("abca"))
#print("ex 2 abd:", example2.n_accepts("abd"))
print(example2.eps_free())
#example3 = fsa(initial={0}, final={2}, transitions={0:{"a":{1}}, 1:{"b":{2}, "*E*":{2}}})
#print(example3.eps_free())

({0: {'a': {1}}, 1: {'b': {2}, 'd': {3}, 'c': {0}}, 2: {'d': {3}, 'c': {0}, 'b': {2}}}, {1, 2})


In [ ]:
test = fsa(initial={1}, final={3}, transitions={1:{"a":{2}}, 2:{"b":{3,4}}, 3:{"b":{3}}, 4:{"c":{3}}})
test2 = fsa(initial={1}, final={3}, transitions={1:{"a":{2}, "*E*": {3}}, 2:{"b":{3,4}}, 3:{"b":{3}, "*E*":{3}}, 4:{"c":{3}}})
test3 = fsa(initial = {1}, final = {3,4}, transitions = {1: {"a":{2}}, 2:{"b":{3},"*E*":{3}}, 3:{"d":{4},"c":{1}}})
test4 = fsa(initial={1}, final={3}, transitions={1:{"a":{2}}, 2:{"b":{3}}, 3:{"b":{3}}})
#test2.mm_con()
#print(test2.nda_da())
#print(test2.n_accepts("b"))
#print(test.n_accepts("b"))
#print(test2.n_accepts("ac"))
print(test3.n_accepts("abc"))
print(test3.n_accepts("ad"))
print(test4.n_accepts("ab"))
print(test2.eps_free())

In [ ]:
#test = fsa(initial = {1}, final = {3}, transitions = {1: {"a":{2}}, 2:{"b":{3},"*E*":{3}}, 3:{"d":{4},"c":{1}}})